In [2]:
import pandas as pd

## Load Alarms with Lavori Programmati

In [2]:
mob = pd.read_parquet("alarms datasets/mob/[new]20230101-20240101_inpas_mob_preprocess__an__last_event__last_event__ext1.parquet")

In [ ]:
adsl = pd.read_parquet("alarms datasets/tx/[new]20230101-20240101_inpas_tx_preprocess__adsl__last_event__last_event__ext1.parquet")
pdh = pd.read_parquet("alarms datasets/tx/[new]20230101-20240101_inpas_tx_preprocess__pdh__last_event__last_event__ext1.parquet")
ptn = pd.read_parquet("alarms datasets/tx/[new]20230101-20240101_inpas_tx_preprocess__ptn__last_event__last_event__ext1.parquet")
sdh = pd.read_parquet("alarms datasets/tx/[new]20230101-20240101_inpas_tx_preprocess__sdh__last_event__last_event__ext1.parquet")

In [12]:
mob = mob[["alarm_id", "std_probable_cause_no", "lp_id", "lp_type"]]
adsl = adsl[["alarm_id", "std_probable_cause_no", "lp_id", "lp_type"]]
pdh = pdh[["alarm_id", "std_probable_cause_no", "lp_id", "lp_type"]]
ptn = ptn[["alarm_id", "std_probable_cause_no", "lp_id", "lp_type"]]
sdh = sdh[["alarm_id", "std_probable_cause_no", "lp_id", "lp_type"]]

In [13]:
alarms_slogan_lavori_programmati = pd.concat([mob, adsl, pdh, ptn, sdh])

In [14]:
alarms_slogan_lavori_programmati.to_parquet("alarms datasets/alarms_slogan_lavori_programmati.parquet")

## Merge Clusters and Alarms with Lavori Programmati

In [33]:
clusters = pd.read_parquet("real-time clusters/20230101-20240101_real_time_clusters_filtered.parquet")

In [35]:
clusters.groupby("cluster_id2").size().sort_values(ascending=False)

cluster_id2
202309181126_19    10079
202308080334_40     7989
202311270204_5      6728
202311270406_7      6611
202310110506_13     6226
                   ...  
202309301148_6         2
202309301148_5         2
202304271552_8         2
202304271552_9         2
202312312358_6         2
Length: 864160, dtype: int64

In [4]:
alarms_slogan_lavori_programmati = pd.read_parquet("alarms datasets/alarms_slogan_lavori_programmati.parquet")

In [5]:
clusters_with_lavori_programmati = pd.merge(clusters, alarms_slogan_lavori_programmati, on="alarm_id", how="left")

In [6]:
clusters_with_lavori_programmati = clusters_with_lavori_programmati[
    ["alarm_id", "cluster_id2", "std_probable_cause_no", "lp_type"]
]

In [7]:
filtered_df = clusters_with_lavori_programmati[clusters_with_lavori_programmati['lp_type'].apply(lambda x: x != 'None')]
filtered_df

,alarm_id,cluster_id2,std_probable_cause_no,lp_type
0,COS1P;136885076,202301010000_16,apparato isolato,None
1,COS1P;136885115,202301010000_16,apparato isolato,None
2,COS1P;136884642,202301010000_25,apparato isolato,None
3,COS1P;136884882,202301010000_25,apparato isolato,None
4,COS1P;136884883,202301010000_25,apparato isolato,None
...,...,...,...,...
8607319,EFM;6591f07dadefac0001744e96,202312312354_7,base station connectivity problem,None
8607320,EFM;6591f08739628d00013784db,202312312354_7,base station connectivity problem,None
8607321,EFM;6591f08cadefac0001744ea1,202312312354_7,base station connectivity problem,None
8607322,EFMTX;6591ee0801021b00010facb4,202312312340_13,fermo totale apparato,None


In [8]:
clusters_with_lavori_programmati

,alarm_id,cluster_id2,std_probable_cause_no,lp_type
0,COS1P;136885076,202301010000_16,apparato isolato,None
1,COS1P;136885115,202301010000_16,apparato isolato,None
2,COS1P;136884642,202301010000_25,apparato isolato,None
3,COS1P;136884882,202301010000_25,apparato isolato,None
4,COS1P;136884883,202301010000_25,apparato isolato,None
...,...,...,...,...
8607319,EFM;6591f07dadefac0001744e96,202312312354_7,base station connectivity problem,None
8607320,EFM;6591f08739628d00013784db,202312312354_7,base station connectivity problem,None
8607321,EFM;6591f08cadefac0001744ea1,202312312354_7,base station connectivity problem,None
8607322,EFMTX;6591ee0801021b00010facb4,202312312340_13,fermo totale apparato,None


In [9]:
unique_lp_type_values = clusters_with_lavori_programmati['lp_type'].apply(lambda x: str(x)).unique()
unique_lp_type_values

array(['None', 'nan', "['APPARATI']", "['CAVO']", "['DWDM/PTN']",
       "['REIX/UPGRADE']", "['CAMBI RELEASE']", "['CAVO' 'DWDM/PTN']",
       "['CAVO' 'APPARATI']"], dtype=object)

In [10]:
clusters_with_lavori_programmati['lp_type'] = clusters_with_lavori_programmati['lp_type'].replace([None], ['None'])

In [11]:
clusters_with_lavori_programmati['lp_type'] = clusters_with_lavori_programmati['lp_type'].str.replace(r"\[|\]|\'", "", regex=True)

In [12]:
unique_lp_type_values = clusters_with_lavori_programmati['lp_type'].apply(lambda x: str(x)).unique()
unique_lp_type_values

array(['None', 'APPARATI', 'CAVO', 'DWDM/PTN', 'REIX/UPGRADE',
       'CAMBI RELEASE', 'CAVO DWDM/PTN', 'CAVO APPARATI'], dtype=object)

## Create a Dataframe with Clusters and "GUASTO CAVO" label

In [13]:
clusters_grouped = clusters_with_lavori_programmati.groupby(["cluster_id2"])

In [23]:
def check_cavo_alarm(group):
    return any(sublist != 'None' for sublist in group['lp_type'].values)

clusters_grouped.apply(check_cavo_alarm).sum()

C:\Users\Nicolò Caradonna\AppData\Local\Temp\ipykernel_11488\1376551276.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  clusters_grouped.apply(check_cavo_alarm).sum()


4191

### Labeling Rule

- GUASTO CAVO = 1 : if at least one alarm inside the cluster has a lp_type == 'CAVO'
- GUASTO CAVO = 0 : if there are no alarms with a lp_type == 'CAVO'
- No labeling (cluster not in the dataframe) : if whole number of alarms in the cluster have a lp_type == 'None' (no lavoro programmato info)

In [24]:
def check_cavo_alarm(group):
    if all(x == 'None' for x in group['lp_type'].values):
        return None
    return any(sublist == 'CAVO' for sublist in group['lp_type'].values if sublist is not None)

temp_df = clusters_grouped.apply(lambda group: pd.Series({
    'cluster_id2': group.name,
    'GUASTO CAVO': check_cavo_alarm(group)
}))

new_df = temp_df.dropna().reset_index(drop=True)

C:\Users\Nicolò Caradonna\AppData\Local\Temp\ipykernel_11488\1928354519.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  temp_df = clusters_grouped.apply(lambda group: pd.Series({


In [25]:
new_df.sort_values(by='GUASTO CAVO', ascending=False)

,cluster_id2,GUASTO CAVO
2095,202306230208_3,True
2644,202308310114_23,True
2630,202308300216_23,True
2631,202308300216_33,True
2632,202308300216_34,True
...,...,...
871,202303231406_21,False
870,202303231406_2,False
869,202303230450_5,False
868,202303230428_4,False


In [26]:
guasto_cavo_count = new_df['GUASTO CAVO'].sum()
guasto_cavo_count

3442

In [27]:
new_df.to_csv("real-time clusters/20230101-20240101_real_time_clusters_filtered_guasto_cavo.csv", index=False)

In [30]:
filtered_clusters = clusters[clusters['cluster_id2'].isin(new_df['cluster_id2'])]

In [32]:
filtered_clusters.groupby("cluster_id2").size().sort_values(ascending=False)

cluster_id2
202310240038_9     3487
202306080506_13    2126
202312050110_59    1609
202311140056_10    1300
202302150200_9     1142
                   ... 
202306060226_3        2
202302230306_6        2
202308100306_11       2
202302230258_13       2
202305240210_15       2
Length: 4191, dtype: int64